## Import Sections

In [22]:
import os
import random

import openai
import pandas as pd
from datasets import load_dataset

import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import COPRO

### Downloading & Exploring Dataset
The dataset can be seen/understood from here: [Dataset Page](https://huggingface.co/datasets/Divyanshu/indicxnli)

In [4]:
# Load Hindi(hi) Dataset. If it's first time, dataset will get downloaded at the location specified against "cache_dir" variable
dataset = load_dataset("Divyanshu/indicxnli", "hi", cache_dir=os.path.join(os.getcwd(), "datasets"))

<b>Viewing one datapoint</b>

In [5]:
split = random.choice(['train', 'test', 'validation'])
index = random.randint(0, 2490)

dataset[split][index]

{'premise': 'उह, मैं अभी भी एक था और केवल नौ दो-दो कि कभी नियामक पर इंजेक्शन सेट किया था।',
 'hypothesis': 'मैं 922 था.',
 'label': 0}

<b>Premise</b>:  This is the initial statement or piece of text that provides factual information or context. It serves as the foundation or given information that we assume to be true for the inference task.

<b>Hypothesis</b>: This is a statement that may or may not follow logically from the premise. It's the claim we want to test against the premise to determine the logical relationship.

<b>Label</b>: This indicates the relationship between the premise and hypothesis:

Integer label `0` if hypothesis `entails` the premise, `2` if hypothesis `negates` the premise and `1` otherwise

### Code

#### Configuring LLM for DSPy

In [6]:
llm = dspy.LM('openai/gpt-4o-mini', api_key = os.getenv('OPENAI_API_KEY'))
dspy.configure(lm=llm)

---
Creating the `dataloader` function to load and return the datapoints as `dspy.Example` object.

#### dspy.Example
The core data type for data in DSPy is `Example`. `Example` represents items in training and test datasets. While doing evaluation & optimization runs, an individual datapoint will be of type `Example`.

`Example` objects have `with_inputs()` method that marks specific fields as inputs. The rest are labels or metadata.

In [7]:
def load_indicxlni(dataset, split="validation", nos_of_points=-1):
    if nos_of_points == -1:
        data_df = pd.DataFrame(dataset[split])
    else:
        data_df = pd.DataFrame(dataset[split]).head(nos_of_points)
    
    label_map = {
        0: "Yes",
        1: "Neutral",
        2: "No"
    }
    
    def as_dspy_example(row):
        return dspy.Example({
            "premise": row['premise'],
            "hypothesis": row['hypothesis'],
            "answer": label_map[row['label']]
        }).with_inputs("premise", "hypothesis")

    return list(data_df.apply(as_dspy_example, axis=1).values)

##### Train | Dev | Test Samples

In [8]:
all_train_set = load_indicxlni(dataset, "train")
all_dev_set = load_indicxlni(dataset, "validation")

random.seed(42)
random.shuffle(all_train_set)
random.shuffle(all_dev_set)

# 50 random train sample and 10 random dev sample
train_set, dev_set = all_train_set[: 50], all_dev_set[:10]

print("Size of training set: ", len(train_set))
print("Size of dev set: ", len(dev_set))

Size of training set:  50
Size of dev set:  10


In [9]:
all_test_set = load_indicxlni(dataset, "test")

random.shuffle(all_test_set)

test_set = all_test_set[:10]
print("Size of test set: ", len(test_set))

Size of test set:  10


### dspy.Signature

`Signature` define the input-output behaviour of LLM calls in a declarative way. Let's define a signature for our dataset where `premise` and `hypothesis` would be inputs to the LLM or prompt and `answer` would be returned by the LLM.

The `doc_string` serves as the prompt to be sent to the LLM with the variables defined in the class as inputs and outputs.

In [11]:
class IndicXLNISignature(dspy.Signature):
    ("""You are given a premise and a hypothesis."""
    """You must indicate with Yes/No/Neutral answer whether we can logically conclude the hypothesis from the premise.""")

    premise = dspy.InputField()
    hypothesis = dspy.InputField()
    answer = dspy.OutputField(desc="Yes or No or Neutral")

Let's use a basic `dspy.Module` to see the prompt created with the above `Signature` class.

In [12]:
basic_prediction = dspy.Predict(IndicXLNISignature)

prediction = basic_prediction(premise = dataset['train'][5]['premise'], hypothesis = dataset['train'][5]['hypothesis'])
print(prediction)

Prediction(
    answer='Yes'
)


The prompt sent or used for the above prediction can be obtained and is this:

In [14]:
basic_prediction.history

[{'prompt': None,
  'messages': [{'role': 'system',
    'content': 'Your input fields are:\n1. `premise` (str): \n2. `hypothesis` (str):\nYour output fields are:\n1. `answer` (str): Yes or No or Neutral\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## premise ## ]]\n{premise}\n\n[[ ## hypothesis ## ]]\n{hypothesis}\n\n[[ ## answer ## ]]\n{answer}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        You are given a premise and a hypothesis.You must indicate with Yes/No/Neutral answer whether we can logically conclude the hypothesis from the premise.'},
   {'role': 'user',
    'content': '[[ ## premise ## ]]\nमेरा वॉकमैन टूट गया तो मैं अब परेशान हूँ.... मुझे बस स्टीरियो को असली जोर से चलाना है\n\n[[ ## hypothesis ## ]]\nमैं परेशान हूं कि मेरा वॉकमैन टूट गया और अब मुझे स्टीरियो को जोर से चलाना पड़ रहा है।\n\nRespond with the corresponding output fields, starting with the field `[[ ## answer ## ]]`, 

---
#### dspy.Module

DSPy Module abstracts prompting technique. We've already seen `dspy.Predict()` module before which is for basic prompting.

We will use advanced prompting technique, Chain Of Thought (COT), for this project, and we will use `dspy.ChainOfThought()` module for it.

In [15]:
class IndicXLNICOT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(IndicXLNISignature)

    def forward(self, premise, hypothesis):
        return self.generate_answer(premise=premise, hypothesis=hypothesis)

Let's see how different this prompt would be as compared to `dspy.Predict()` module.

In [16]:
cot_zeroshot = IndicXLNICOT()

cot_prediction = cot_zeroshot(premise = dataset['train'][7]['premise'], hypothesis = dataset['train'][7]['hypothesis'])
print(cot_prediction)

Prediction(
    reasoning='प्रस्तावना में केवल यह कहा गया है कि स्लेट ने जैक्सन के निष्कर्षों को पढ़ा है, लेकिन यह नहीं बताया गया है कि स्लेट की जैक्सन के निष्कर्षों पर कोई राय थी या नहीं। इसलिए, हम यह नहीं कह सकते कि स्लेट की जैक्सन के निष्कर्षों पर कोई राय थी।',
    answer='No'
)


In [17]:
cot_zeroshot.history[-1]

{'prompt': None,
 'messages': [{'role': 'system',
   'content': 'Your input fields are:\n1. `premise` (str): \n2. `hypothesis` (str):\nYour output fields are:\n1. `reasoning` (str): \n2. `answer` (str): Yes or No or Neutral\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## premise ## ]]\n{premise}\n\n[[ ## hypothesis ## ]]\n{hypothesis}\n\n[[ ## reasoning ## ]]\n{reasoning}\n\n[[ ## answer ## ]]\n{answer}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        You are given a premise and a hypothesis.You must indicate with Yes/No/Neutral answer whether we can logically conclude the hypothesis from the premise.'},
  {'role': 'user',
   'content': '[[ ## premise ## ]]\n(स्लेट के जैक्सन के निष्कर्षों को पढ़ने के लिए पढ़ें।)\n\n[[ ## hypothesis ## ]]\nजैक्सन के निष्कर्षों पर स्लेट की राय थी।\n\nRespond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## answ

---
#### Metrics & Evaluator

In DSPy, metric is a function that will take examples from your data and the output of LLM and return a score that quantifies how good the output is.

For simple classification or short-form QA tasks, this could be just "accuracy", or "exact match" or "F1-Score". For long-form outputs, metric could be a smaller DSPy program that checks multiple properties of the output(using AI feedback from LLMs).

Simpler metrics are already defined in DSPY:
- `dspy.evaluate.metrics.answer_exact_match`
- `dspy.evaluate.metrics.answer_passage_match`

The metrics could be more complex, e.g. check for multiple properties. For that, write your own function.

In [18]:
indicxlni_accuracy = dspy.evaluate.metrics.answer_exact_match

Once metric is decided/defined, evaluations can be done either in a simple Python loop:

```
scores = []
for x in devset:
    pred = program(**x.inputs())
    score = metric(x, pred)
    scores.append(score)
```

or can use built-in utilites in `Evaluate` utilities.

In [20]:
evaluator = Evaluate(devset=test_set, num_threads=1, display_progress=True, display_table=len(test_set))

In [21]:
evaluator(cot_zeroshot, metric=indicxlni_accuracy)

Average Metric: 7.00 / 10 (70.0%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 99.49it/s]

2025/06/21 13:56:28 INFO dspy.evaluate.evaluate: Average Metric: 7 / 10 (70.0%)


,premise,hypothesis,example_answer,reasoning,pred_answer,answer_exact_match
0,सुरक्षा सुनिश्चित करने के लिए जनरेटरों को बंद करना पड़ा और लिफ्ट ब...,जनरेटर एक सुरक्षा जोखिम थे।,Yes,प्रस्तावना में कहा गया है कि सुरक्षा सुनिश्चित करने के लिए जनरेटरो...,Yes,✔️ [True]
1,"भगवान का उल्लेख केवल प्रकृति के भगवान के रूप में किया जाता है, जिस...",लोग अलग-अलग होंगे लेकिन समान होंगे।,Yes,प्रस्तावना में कहा गया है कि सभी लोगों को राष्ट्रों के समुदाय में ...,Yes,✔️ [True]
2,"अप्रैल १८६५ में उस व्यक्ति की हत्या के साथ, जिसने विचारों के एक नए...",इस हमले को रूस ने अंजाम दिया था।,Neutral,प्रस्तावना में यह कहा गया है कि एक व्यक्ति की हत्या अप्रैल 1865 मे...,No,
3,"दूसरे शब्दों में, क्या होता है कुछ की तरह है एक जादूगर के अब-तुम-अ...",जो हो रहा है वह हैरान करने वाला है।,Yes,"प्रस्तावना में एक जादूगर के कार्यों का उल्लेख किया गया है, जो आमतौ...",Neutral,
4,"आप जानते हैं, आप नहीं बच सकते, आप जीवित नहीं रह सकते अगर आपके पास ...",आपको 5000 फीट से अधिक काउंटर प्रेशर की जरूरत होती है।,Neutral,प्रस्तावना में कहा गया है कि ऊंचाइयों पर सांस लेने के लिए काउंटर प...,Neutral,✔️ [True]
5,"एक साधारण उदाहरण के रूप में, मान लीजिए कि काम साझा करने की लागत 10...",आप अनुमान लगा सकते हैं कि काम साझा करने की लागत मूल डाक से कम है।,Yes,प्रस्तावना में कहा गया है कि काम साझा करने की लागत 10a है और मूल ड...,Yes,✔️ [True]
6,आजादी के बाद यहां संसद बनाने की योजना शून्य हो गई।,इस स्थान के लिए संसद पर दृढ़ता से विचार किया गया था।,Neutral,प्रस्तावना में कहा गया है कि आजादी के बाद संसद बनाने की योजना शून्...,No,
7,हाँ मैं उसे सुन सकता हूँ,मैं उसे सुन नहीं सकता.,No,"The premise states ""हाँ मैं उसे सुन सकता हूँ,"" which translates to...",No,✔️ [True]
8,"इस समूह में 23वां संशोधन भी है, जो कोलंबिया जिले में अन्यथा योग्य ...",23वें संशोधन में कहा गया है कि अगर आप कैलिफोर्निया में रहते हैं तो...,No,23वें संशोधन का संदर्भ कोलंबिया जिले में अन्यथा योग्य नागरिकों को ...,No,✔️ [True]
9,"रिसॉर्ट के केंद्र में, भीतरी लैगून के आश्रय वाले पानी में, डॉल्फ़ि...",आप रिसॉर्ट में डोफिन के साथ तैर सकते हैं।,Yes,प्रस्तावना में कहा गया है कि रिसॉर्ट में डॉल्फ़िन कार्यक्रम के साथ...,Yes,✔️ [True]


70.0

---
### Optimization

In [23]:
copro_optimizer = COPRO(metric=indicxlni_accuracy, depth=2)

In [24]:
kwargs = dict(num_threads=64, display_progress=True)

cot_copro = copro_optimizer.compile(cot_zeroshot, trainset=train_set, eval_kwargs=kwargs)

2025/06/21 14:36:16 INFO dspy.teleprompt.copro_optimizer: Iteration Depth: 1/2.
2025/06/21 14:36:17 INFO dspy.teleprompt.copro_optimizer: At Depth 1/2, Evaluating Prompt Candidate #1/10 for Predictor 1 of 1.


Average Metric: 35.00 / 50 (70.0%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 12.05it/s]

2025/06/21 14:36:21 INFO dspy.evaluate.evaluate: Average Metric: 35 / 50 (70.0%)
2025/06/21 14:36:21 INFO dspy.teleprompt.copro_optimizer: At Depth 1/2, Evaluating Prompt Candidate #2/10 for Predictor 1 of 1.



Average Metric: 34.00 / 50 (68.0%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:03<00:00, 15.04it/s]

2025/06/21 14:36:25 INFO dspy.evaluate.evaluate: Average Metric: 34 / 50 (68.0%)
2025/06/21 14:36:25 INFO dspy.teleprompt.copro_optimizer: At Depth 1/2, Evaluating Prompt Candidate #3/10 for Predictor 1 of 1.


...

Average Metric: 39.00 / 50 (78.0%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:05<00:00,  9.02it/s]

2025/06/21 14:38:02 INFO dspy.evaluate.evaluate: Average Metric: 39 / 50 (78.0%)


2025/06/21 14:38:02 INFO dspy.teleprompt.copro_optimizer: At Depth 2/2, Evaluating Prompt Candidate #10/10 for Predictor 1 of 1.


Average Metric: 37.00 / 50 (74.0%): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:04<00:00, 11.13it/s]

2025/06/21 14:38:07 INFO dspy.evaluate.evaluate: Average Metric: 37 / 50 (74.0%)


#### Testing optimized prompt on test set

In [25]:
evaluator(cot_copro, metric=indicxlni_accuracy)

Average Metric: 7.00 / 10 (70.0%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:20<00:00,  2.07s/it]

2025/06/21 14:40:34 INFO dspy.evaluate.evaluate: Average Metric: 7 / 10 (70.0%)


,premise,hypothesis,example_answer,reasoning,pred_answer,answer_exact_match
0,सुरक्षा सुनिश्चित करने के लिए जनरेटरों को बंद करना पड़ा और लिफ्ट ब...,जनरेटर एक सुरक्षा जोखिम थे।,Yes,प्रस्तावना में कहा गया है कि सुरक्षा सुनिश्चित करने के लिए जनरेटरो...,Yes,✔️ [True]
1,"भगवान का उल्लेख केवल प्रकृति के भगवान के रूप में किया जाता है, जिस...",लोग अलग-अलग होंगे लेकिन समान होंगे।,Yes,प्रस्तावना में कहा गया है कि सभी लोगों को राष्ट्रों के समुदाय में ...,Yes,✔️ [True]
2,"अप्रैल १८६५ में उस व्यक्ति की हत्या के साथ, जिसने विचारों के एक नए...",इस हमले को रूस ने अंजाम दिया था।,Neutral,प्रस्तावना में यह कहा गया है कि एक व्यक्ति की हत्या हुई थी जिसने न...,No,
3,"दूसरे शब्दों में, क्या होता है कुछ की तरह है एक जादूगर के अब-तुम-अ...",जो हो रहा है वह हैरान करने वाला है।,Yes,"प्रस्तावना में एक जादूगर के कार्यों का उल्लेख किया गया है, जो दर्श...",Neutral,
4,"आप जानते हैं, आप नहीं बच सकते, आप जीवित नहीं रह सकते अगर आपके पास ...",आपको 5000 फीट से अधिक काउंटर प्रेशर की जरूरत होती है।,Neutral,प्रस्तावना में कहा गया है कि ऊंचाइयों पर जीवित रहने के लिए काउंटर ...,Neutral,✔️ [True]
5,"एक साधारण उदाहरण के रूप में, मान लीजिए कि काम साझा करने की लागत 10...",आप अनुमान लगा सकते हैं कि काम साझा करने की लागत मूल डाक से कम है।,Yes,प्रस्तावना में कहा गया है कि काम साझा करने की लागत 10a है और मूल ड...,Yes,✔️ [True]
6,आजादी के बाद यहां संसद बनाने की योजना शून्य हो गई।,इस स्थान के लिए संसद पर दृढ़ता से विचार किया गया था।,Neutral,प्रस्तावना में कहा गया है कि आजादी के बाद संसद बनाने की योजना शून्...,No,
7,हाँ मैं उसे सुन सकता हूँ,मैं उसे सुन नहीं सकता.,No,"The premise states ""हाँ मैं उसे सुन सकता हूँ,"" which translates to...",No,✔️ [True]
8,"इस समूह में 23वां संशोधन भी है, जो कोलंबिया जिले में अन्यथा योग्य ...",23वें संशोधन में कहा गया है कि अगर आप कैलिफोर्निया में रहते हैं तो...,No,23वां संशोधन कोलंबिया जिले में अन्यथा योग्य नागरिकों को राष्ट्रपति...,No,✔️ [True]
9,"रिसॉर्ट के केंद्र में, भीतरी लैगून के आश्रय वाले पानी में, डॉल्फ़ि...",आप रिसॉर्ट में डोफिन के साथ तैर सकते हैं।,Yes,प्रस्तावना में कहा गया है कि रिसॉर्ट में डॉल्फ़िन कार्यक्रम के साथ...,Yes,✔️ [True]


70.0

In [26]:
cot_copro.history[-1]

{'prompt': None,
 'messages': [{'role': 'system',
   'content': 'Your input fields are:\n1. `premise` (str): \n2. `hypothesis` (str):\nYour output fields are:\n1. `reasoning` (str): \n2. `answer` (str): Yes or No or Neutral\nAll interactions will be structured in the following way, with the appropriate values filled in.\n\n[[ ## premise ## ]]\n{premise}\n\n[[ ## hypothesis ## ]]\n{hypothesis}\n\n[[ ## reasoning ## ]]\n{reasoning}\n\n[[ ## answer ## ]]\n{answer}\n\n[[ ## completed ## ]]\nIn adhering to this structure, your objective is: \n        Carefully examine the relationship between the given premise and hypothesis. Your task is to ascertain if the hypothesis can logically be inferred from the premise. Retreating into logic:, respond affirmatively with "Yes" if the hypothesis follows validly; denote doubt if "Neutral" applies where the hypothesis doesn\'t contradict but isn\'t confirmed either, or decisively declare "No" if it incoherently counters the premise.'},
  {'role': 'user

In [30]:
print(cot_copro.history[-1]['messages'][0]['content'])

Your input fields are:
1. `premise` (str): 
2. `hypothesis` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str): Yes or No or Neutral
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## premise ## ]]
{premise}

[[ ## hypothesis ## ]]
{hypothesis}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Carefully examine the relationship between the given premise and hypothesis. Your task is to ascertain if the hypothesis can logically be inferred from the premise. Retreating into logic:, respond affirmatively with "Yes" if the hypothesis follows validly; denote doubt if "Neutral" applies where the hypothesis doesn't contradict but isn't confirmed either, or decisively declare "No" if it incoherently counters the premise.


### PROMPT CHANGES SHOWN

#### Initial Prompt
This is the prompt that we passed in `Signature`::

----------------------------------------------------------------------------
Your input fields are:
1. `premise` (str): 
2. `hypothesis` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str): Yes or No or Neutral
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## premise ## ]]
{premise}

[[ ## hypothesis ## ]]
{hypothesis}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are given a premise and a hypothesis.You must indicate with Yes/No/Neutral answer whether we can logically conclude the hypothesis from the premise.

#### Prompt After Optimization
Your input fields are:
1. `premise` (str): 
2. `hypothesis` (str):
Your output fields are:
1. `reasoning` (str): 
2. `answer` (str): Yes or No or Neutral
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## premise ## ]]
{premise}

[[ ## hypothesis ## ]]
{hypothesis}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Carefully examine the relationship between the given premise and hypothesis. Your task is to ascertain if the hypothesis can logically be inferred from the premise. Retreating into logic:, respond affirmatively with "Yes" if the hypothesis follows validly; denote doubt if "Neutral" applies where the hypothesis doesn't contradict but isn't confirmed either, or decisively declare "No" if it incoherently counters the premise.